# Logger leveraging Xterm
The ColoredFormatter used Pygment tokens so custom styles can be easily made.

In [ ]:
import logging
from wxyz.lab.widget_dock import DockBox, DockPop, W, T
from wxyz.lab.widget_logger import XStream, ColoredFormatter, Style, DarkLogStyle
from pygments.token import Comment, Name, Generic, String
from wxyz.html.widget_color import AlphaColorPicker
from pygments.formatters import TerminalTrueColorFormatter
from pygments.style import _ansimap

def simple_logger_demo():
    stream = XStream()

    logger = logging.getLogger("simple_demo")
    handler = logging.StreamHandler(stream=stream)
    colorer = ColoredFormatter(
        fmt="%(asctime)s-%(name)s:%(levelname)s- %(message)s",
        datefmt='%H:%M:%S',
    )
    logger.setLevel(logging.DEBUG)
    handler.setFormatter(colorer)
    logger.addHandler(handler)

    logger.debug("debug message")
    logger.info("info message")
    logger.warn("warn message")
    logger.error("error message")
    logger.critical("critical message")

    return logger, stream.terminal

if __name__ == "__main__":
    logger, terminal = simple_logger_demo()
    display(terminal)
    
    try:
        1/0
    except:
        logger.exception("Some bad thing happened")
        

Exceptions logged and colored using IPython verbose traceback

# Example showing how to style new messages

In [ ]:
def to_styledict(style:str)->dict:
    is_italic = "italic" in style
    is_bold = "bold" in style
    color = style.replace("italic", "").replace("bold", "").strip()
    if "ansi" in color:
        color = _ansimap.get(f"{color}", color)
    return {
        "is_italic": is_italic,
        "is_bold": is_bold,
        "color":color,
    }

def to_stylestring(style:dict)->str:
    attrs = []
    if style.get("is_italic"):
        attrs.append("italic")
    if style.get("is_bold"):
        attrs.append("bold")
    attrs.append(style.get("color"))
    return " ".join(attrs)


def make_a_fancy_logging_demo():
    stream = XStream()

    logger = logging.getLogger("fancy_demo")
    handler = logging.StreamHandler(stream=stream)
    colorer = ColoredFormatter(
        fmt="%(asctime)s-%(name)s:%(levelname)s- %(message)s",
        datefmt='%H:%M:%S',
    )
    logger.setLevel(logging.DEBUG)
    handler.setFormatter(colorer)
    logger.addHandler(handler)

    # Message creation widgets
    options = [(lvl, getattr(logger, lvl.lower())) for lvl in ["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"]]
    level = W.Dropdown(options=options)
    msg_input = W.Textarea(description="Message")
    log_btn = W.Button(description="Log")

    msg_creation = W.VBox([level, msg_input, log_btn])

    styling = {key:to_styledict(value) for key, value in colorer._formatter.style.styles.items() if value}
    token_map = {
        "Logger Name": Name.Namespace,
        "Time stamp": Name.Tag,
        "Level": Name.Decorator,
        "Debug Msg": Generic.Inserted,
        "Info Msg": Generic.Output,
        "Warning Msg": Generic.Prompt,
        "Error Msg": Generic.Error,
        "Critical Msg": Generic.Strong,
        "Misc Formatting Characters": String.Delimiter,
    }

    styling_widgets = {}
    for name, token in token_map.items():
        sty = styling.get(token,{})
        color_picker = AlphaColorPicker(
            description=name, 
            value=f"#{sty.get('color', '000000')}"
        )
        italic_cb = W.Checkbox(
            description="Italic", 
            value=sty.get("is_italic", False),
            layout=W.Layout(width='150px'),
        ).add_class("hideLabel")
        bold_cb = W.Checkbox(
            description="Bold", 
            value=sty.get("is_bold", False),
            layout=W.Layout(width='150px'),
        ).add_class("hideLabel")

        styling_widgets[token]=W.HBox([color_picker, italic_cb, bold_cb])

    def build_style_cls():
        style_dict = {}
        for token, widgets in styling_widgets.items():
            style_dict[token] = to_stylestring({key:(w.value) for key, w in zip(["color", "is_italic", "is_bold"], widgets.children)})
        class StyleCls(Style):
            styles = style_dict
        return StyleCls

    css_mod = W.HTML(value="<style>.hideLabel .widget-label {display:none;} .style_pickers {min-width:440px;}<style>")
    style_pickers = W.VBox([css_mod] + list(styling_widgets.values())).add_class("style_pickers")

    def logit(btn):
        colorer._formatter = TerminalTrueColorFormatter(style=build_style_cls())
        level.value(msg_input.value)
    log_btn.on_click(logit)
    
    msg_creation.add_traits(description=T.Unicode("Log Message").tag(sync=True))
    style_pickers.add_traits(description=T.Unicode("Message Style").tag(sync=True))
    stream.terminal.description = "Logging Terminal"
    box = DockBox([style_pickers, msg_creation, stream.terminal], layout=dict(height="400px"), dock_layout={
        
    })
    return box

if __name__ == "__main__":
    display(make_a_fancy_logging_demo())